**In this Notebook we will explore a "USA_flights" dataset using pandas, and perform various data analysis tasks.**

In [ ]:
import numpy as np
import pandas as pd
# pip install seaborn 
import seaborn as sns
import matplotlib.pyplot as plt

* The data [archive] was downloaded from (http://stat-computing.org/dataexpo/2009/2008.csv.bz2) (Archived ~ 114 Mb, unzipped - ~ 690 Mb). No need to unzip - pandas can unbzip on the fly.
* The dataset has information about carriers and flights between US airports during the year 2008. 
* Column description is available [here](http://www.transtats.bts.gov/Fields.asp?Table_ID=236). Visit this site to find ex. meaning of flight cancellation codes.

**Reading data into memory and creating a Pandas _DataFrame_ object**

(This may take a while)

We are not going to read in the whole dataset. In order to reduce memory footprint, we instead load only needed columns and cast them suitable data types.

In [ ]:
dtype = {'DayOfWeek': np.uint8, 'DayofMonth': np.uint8, 'Month': np.uint8 , 'Cancelled': np.uint8, 
         'Year': np.uint16, 'FlightNum': np.uint16 , 'Distance': np.uint16, 
         'UniqueCarrier': str, 'CancellationCode': str, 'Origin': str, 'Dest': str,
         'ArrDelay': np.float16, 'DepDelay': np.float16, 'CarrierDelay': np.float16,
         'WeatherDelay': np.float16, 'NASDelay': np.float16, 'SecurityDelay': np.float16,
         'LateAircraftDelay': np.float16, 'DepTime': np.float16}

In [ ]:
%%time
# change the path if needed
path = '../../data/2008.csv.bz2'
flights_df = pd.read_csv(path, usecols=dtype.keys(), dtype=dtype)

**Check the number of rows and columns and print column names.**

In [ ]:
print(flights_df.shape)
print(flights_df.columns)

**Print first 5 rows of the dataset.**

In [ ]:
flights_df.head()

**Transpose the frame to see all features at once.**

In [ ]:
flights_df.head().T

**Examine data types of all features and total dataframe size in memory.**

In [ ]:
flights_df.info()

**Get basic statistics of each feature.**

In [ ]:
flights_df.describe().T

**Count unique Carriers and plot their relative share of flights:**

In [ ]:
flights_df['UniqueCarrier'].nunique()

In [ ]:
flights_df.groupby('UniqueCarrier').size().plot(kind='bar');

**We can also _group by_ category/categories in order to calculate different aggregated statistics.**

**For example, finding top-3 flight codes, that have the largest total distance travelled in year 2008.**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])['Distance'].sum().sort_values(ascending=False).iloc[:3]

**Another way:**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])\
  .agg({'Distance': [np.mean, np.sum, 'count'],
        'Cancelled': np.sum})\
  .sort_values(('Distance', 'sum'), ascending=False)\
  .iloc[0:3]

**Number of flights by days of week and months:**

In [ ]:
pd.crosstab(flights_df.Month, flights_df.DayOfWeek)

**It can also be handy to color such tables in order to easily notice outliers:**

In [ ]:
plt.imshow(pd.crosstab(flights_df.Month, flights_df.DayOfWeek),
           cmap='seismic', interpolation='none');

**Flight distance histogram:**

In [ ]:
flights_df.hist('Distance', bins=20);

**Making a histogram of flight frequency by date.**

In [ ]:
flights_df['Date'] = pd.to_datetime(flights_df.rename(columns={'DayofMonth': 'Day'})[['Year', 'Month', 'Day']])

In [ ]:
num_flights_by_date = flights_df.groupby('Date').size()

In [ ]:
num_flights_by_date.plot();

**We can see a weekly pattern above, and below?**

In [ ]:
num_flights_by_date.rolling(window=7).mean().plot();

**With the following code we will find the top-10 carriers in terms of the number of completed flights (_UniqueCarrier_ column).**

In [ ]:
final = flights_df.groupby('UniqueCarrier', as_index=False)["DepTime"].count()
final.sort_values(by='DepTime', ascending=False).head(10)

**With the following code we will find what is the most frequent reason for flight cancellation?(We'll use this [link](https://www.transtats.bts.gov/Fields.asp?Table_ID=236) to translate codes into reasons)**

In [ ]:
final2 = flights_df.groupby('CancellationCode', as_index=False).count()
final2.sort_values(by='Year', ascending=False).head()

**With the following code we will find which route is the most frequent, in terms of the number of flights.**

In [ ]:
routes = flights_df['routes'] = flights_df['Origin'] + ' ' + flights_df['Dest'] 
routes.head()

In [ ]:
routes.value_counts().head()

**With the following code we will find the top-5 delayed routes (count how many times they were delayed on departure).**

In [ ]:
flights_df.count()
delayed=flights_df[flights_df['DepDelay']>0]
delayed['routes'] = delayed['Origin'] + ' ' + delayed['Dest'] 
top5delayed=delayed.groupby('routes', as_index=False)['Dest'].count().sort_values(by='Dest', ascending=False).head()
flights_df['routes'] = flights_df['Origin'] + ' ' + flights_df['Dest']
flightsontop5dr=pd.merge(flights_df,top5delayed,how='inner',on='routes')
flightsontop5dr[flightsontop5dr.WeatherDelay.notnull() & flightsontop5dr.WeatherDelay>0].count()

 **With the following code we will find the hourly distribution of departure times. For that, we will create a new series from DepTime, removing missing values.**

In [ ]:
deptimenew = flights_df['DepTime']
deptimenew.dropna(inplace = True)
len(deptimenew)
dpn = deptimenew.floordiv(100)
dpn.hist(bins = 50)

**With the following code we will find how the number of flights changes through time (on the daily/weekly/monthly basis) and interpret the findings.**

In [ ]:
dom = flights_df.groupby('DayofMonth').count()
dom['Month'].plot()
plt.show()
dm = flights_df.groupby('Month').count()
dm['DayofMonth'].plot()
plt.show()
dm = flights_df.groupby('DayOfWeek').count()
dm['DayofMonth'].plot()

**With the following code we will examine the distribution of cancellation reasons with time. We will make a bar plot of cancellation reasons aggregated by months.**

In [ ]:
flights_df.groupby(['CancellationCode']).count()
# flights_df.groupby(['Month', 'CancellationCode']).count()

**With the following code we will find which month has the greatest number of cancellations due to Carrier.**

In [ ]:
dm = flights_df[(flights_df['CancellationCode'] == 'A')]
dm1 = dm.groupby('Month', as_index=False).count()
dm1.sort_values(by='Year', ascending=False).head()

**With the following code we will indentify the carrier with the greatest number of cancellations due to carrier in the corresponding month from the previous exercise.**

In [ ]:
dm2 = flights_df[(flights_df['CancellationCode'] == 'A') & (flights_df['Month'] == 4)]
dm3 = dm2.groupby('UniqueCarrier', as_index=False).count()
dm3.sort_values(by='Year', ascending=False).head()

**With the following code we will examine the median arrival and departure delays (in time) by carrier. We'll aslo find which carrier has the lowest median delay time for both arrivals and departures.** 

In [ ]:
deparr = pd.concat([flights_df.DepDelay, flights_df.ArrDelay, flights_df.UniqueCarrier], axis=1)
deparrmedian = deparr[(deparr['DepDelay'] > 0) & (deparr['ArrDelay'] > 0)]
dp = deparrmedian.sort_values(by='DepDelay', ascending = True)
bx = dp.groupby('UniqueCarrier').median().sort_values(by='DepDelay', ascending = True)
bx
# sns.boxplot(bx)